In [24]:
from ultralytics import YOLO
model = YOLO("yolov8n.pt")
import os
results = model("meeting.jpg", task="detect",save=True, conf=0.5, project="coba", name="aa")


image 1/1 d:\YoloDemo\meeting.jpg: 448x640 7 persons, 1 chair, 1 potted plant, 2 laptops, 49.0ms
Speed: 5.0ms preprocess, 49.0ms inference, 2.1ms postprocess per image at shape (1, 3, 448, 640)
Results saved to coba\aa


yolo task=detect mode=predict model=yolov8n.pt source=marathon.mp4 conf=0.5 show=True

In [ ]:
!ffmpeg -i {"runs\detect\predict3\marathon.avi"} -vcodec libx264 {r"runs\detect\predict3\final.mp4"}

In [ ]:
from datetime import datetime as dt

def predict_process(model_path, target):
    predict = dt.now().strftime('%Y-%m-%d-%H%M%S')
    model = YOLO(model_path)
    results = model(target, task="detect", save=True, conf=0.5, project="detect", name=predict)
    


In [1]:
from ultralytics import YOLO

model = YOLO("model\yolov8n.pt")

model.export(format="onnx", imgsz=[640,640], opset=12)

Ultralytics YOLOv8.2.2  Python-3.10.0 torch-2.2.1+cu121 CPU (12th Gen Intel Core(TM) i7-12700H)
YOLOv8n summary (fused): 168 layers, 3151904 parameters, 0 gradients, 8.7 GFLOPs

PyTorch: starting from 'model\yolov8n.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 84, 8400) (6.2 MB)
requirements: Ultralytics requirement ['onnx>=1.12.0'] not found, attempting AutoUpdate...
   ---------------------------------------- 14.4/14.4 MB 9.0 MB/s eta 0:00:00
   ---------------------------------------- 420.9/420.9 kB 4.4 MB/s eta 0:00:00

requirements: AutoUpdate success  13.4s, installed 1 package: ['onnx>=1.12.0']
requirements:  Restart runtime or rerun command for updates to take effect


ONNX: starting export with onnx 1.16.0 opset 12...
ONNX: export success  14.8s, saved as 'model\yolov8n.onnx' (12.2 MB)

Export complete (18.1s)
Results saved to D:\YoloDemo\model
Predict:         yolo predict task=detect model=model\yolov8n.onnx imgsz=640  
Validate:        yolo val task=de

'model\\yolov8n.onnx'

In [2]:
import cv2
import numpy as np

net = cv2.dnn.readNetFromONNX("model\yolov8n.onnx")
INPUT_WIDTH = 640
INPUT_HEIGHT = 640

image = cv2.imread('meeting.jpg')
blob = cv2.dnn.blobFromImage(image, 1/255.0, (INPUT_WIDTH, INPUT_HEIGHT), swapRB=True, crop=False)
net.setInput(blob)

output = net.forward()
output = output.transpose((0, 2, 1))


# Extract output detection
class_ids, confs, boxes = list(), list(), list()

image_height, image_width, _ = image.shape
x_factor = image_width / INPUT_WIDTH
y_factor = image_height / INPUT_HEIGHT

rows = preds[0].shape[0]

for i in range(rows):
    row = preds[0][i]
    conf = row[4]
    
    classes_score = row[4:]
    _,_,_, max_idx = cv2.minMaxLoc(classes_score)
    class_id = max_idx[1]
    if (classes_score[class_id] > .25):
        confs.append(conf)
        label = int(class_id)
        class_ids.append(label)
        
        #extract boxes
        x, y, w, h = row[0].item(), row[1].item(), row[2].item(), row[3].item() 
        left = int((x - 0.5 * w) * x_factor)
        top = int((y - 0.5 * h) * y_factor)
        width = int(w * x_factor)
        height = int(h * y_factor)
        box = np.array([left, top, width, height])
        boxes.append(box)
        
r_class_ids, r_confs, r_boxes = list(), list(), list()

indexes = cv2.dnn.NMSBoxes(boxes, confs, 0.25, 0.45) 
for i in indexes:
    r_class_ids.append(class_ids[i])
    r_confs.append(confs[i])
    r_boxes.append(boxes[i])



for i in indexes:
    box = boxes[i]
    left = box[0]
    top = box[1]
    width = box[2]
    height = box[3]
    conf = confs[i]
    
    cv2.rectangle(image, (left, top), (left + width, top + height), (0,255,0), 3)

cv2.imwrite("your_result_image.jpg", image)

NameError: name 'preds' is not defined

In [7]:
from ultralytics import YOLO
# Load the exported ONNX model
onnx_model = YOLO('model\yolov8n.onnx')

# Run inference
results = onnx_model('meeting.jpg', task="detect",save=True, conf=0.5, project="coba", name="aa", device='cpu')

WARNING  Unable to automatically guess model task, assuming 'task=detect'. Explicitly define task for your model, i.e. 'task=detect', 'segment', 'classify','pose' or 'obb'.
Loading model\yolov8n.onnx for ONNX Runtime inference...

image 1/1 d:\YoloDemo\meeting.jpg: 640x640 7 persons, 1 chair, 1 potted plant, 2 laptops, 1 vase, 172.4ms
Speed: 3.0ms preprocess, 172.4ms inference, 3.0ms postprocess per image at shape (1, 3, 640, 640)
Results saved to coba\aa8
